# ENSO Region Variability

This notebook plots basic ENSO variability.

### MAR Configuration

In [11]:
import os
import datetime
import doralite
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import doralite
import glob
import subprocess
import momlevel
import itertools
import xwavelet as xw
import momgrid as mg

from datetime import timedelta

print("numpy version  : ", np.__version__)
print("pandas version : ", pd.__version__)
print("xarray version : ", xr.__version__)

config = {
    "startyr": "1",
    "endyr": "60",
    "dora_id": "odiv-413",
    "pathPP": "",
}

for k, v in config.items():
    config[k] = (
        os.environ[f"MAR_{k.upper()}"]
        if f"MAR_{k.upper()}" in os.environ.keys()
        else v
    )

numpy version  :  1.23.5
pandas version :  2.1.1
xarray version :  2023.10.1


In [13]:
import momgrid
os.environ["MOMGRID_WEIGHTS_DIR"] = "/nbhome/John.Krasting/grid_weights"

MAR will pass for environment variables to the script when running via the web engine:

* `MAR_STARTYR`: Beginning year of analysis from model
* `MAR_ENDYR`: Ending year of analysis from model
* `MAR_DORA_ID`: The experiment ID in the database
* `MAR_PATHPP`: The top-level path to the post-processing experiment directory of the experiment

The block below will use values passed in by Dora but default to the values defined above in `config`. This is useful for interactive usage and debugging.

If executed from Dora, there will also be a `DORA_EXECUTE` variable that is set.

In [4]:
# Define some local variables. These are taken from the doralite object
# or they can be defined locally

# 1. List of all experiments to consider
experiments = [doralite.dora_metadata(x) for x in config["dora_id"].split(",")]

# Define start and end years
start = int(config["startyr"])
end = int(config["endyr"])

### Load Files

In [5]:
def is_in_range(file,start,end):
    start = int(start)
    end = int(end)
    target = set(list(range(start,end+1)))
    fname = os.path.basename(file)
    times = fname.split(".")[1]
    times = times.split("-")
    times = [int(x[0:4]) for x in times]
    candidate = set(list(range(times[0],times[1]+1)))
    return len(candidate.intersection(target)) > 0

In [6]:
datafiles = []
staticfiles = []

for experiment in experiments:
    _pathPP = experiment["pathPP"]
    
    component = "ocean_monthly"
    static = f"{component}/{component}.static.nc"
    varname = "tos"
    
    chunk = "5yr"
    filelist = sorted(glob.glob(f"{_pathPP}{component}/ts/monthly/{chunk}/{component}.*.{varname}.nc"))
    
    filelist = [x for x in filelist if is_in_range(x,start,end)]
    _staticfile = f"{_pathPP}/{static}"
    
    staticfiles.append(_staticfile)
    datafiles.append(filelist)

In [7]:
allfiles = sorted(list(itertools.chain(*datafiles)) + staticfiles)
_ = [print(x) for x in allfiles]

/archive/John.Krasting/fre/FMS2024.02_OM5_20240724/CM4.5v01_om5b06_piC_noBLING/gfdl.ncrc5-intel23-prod-openmp/pp//ocean_monthly/ocean_monthly.static.nc
/archive/John.Krasting/fre/FMS2024.02_OM5_20240724/CM4.5v01_om5b06_piC_noBLING/gfdl.ncrc5-intel23-prod-openmp/pp/ocean_monthly/ts/monthly/5yr/ocean_monthly.000101-000512.tos.nc
/archive/John.Krasting/fre/FMS2024.02_OM5_20240724/CM4.5v01_om5b06_piC_noBLING/gfdl.ncrc5-intel23-prod-openmp/pp/ocean_monthly/ts/monthly/5yr/ocean_monthly.000601-001012.tos.nc
/archive/John.Krasting/fre/FMS2024.02_OM5_20240724/CM4.5v01_om5b06_piC_noBLING/gfdl.ncrc5-intel23-prod-openmp/pp/ocean_monthly/ts/monthly/5yr/ocean_monthly.001101-001512.tos.nc
/archive/John.Krasting/fre/FMS2024.02_OM5_20240724/CM4.5v01_om5b06_piC_noBLING/gfdl.ncrc5-intel23-prod-openmp/pp/ocean_monthly/ts/monthly/5yr/ocean_monthly.001601-002012.tos.nc
/archive/John.Krasting/fre/FMS2024.02_OM5_20240724/CM4.5v01_om5b06_piC_noBLING/gfdl.ncrc5-intel23-prod-openmp/pp/ocean_monthly/ts/monthly/5y

In [8]:
if not "DORA_EXECUTE" in os.environ.keys():
    print("Calling dmget on files ...")
    cmd = ["dmget"]+allfiles
    _ = subprocess.check_output(cmd)

Calling dmget on files ...


### Calculations and Plots from CM4X Notebook

In [33]:
# Open dataset
ds = mg.Gridset(filelist).data

# Nino3.4 SST
nino34 = mg.geoslice.geoslice(ds.tos,y=(-5,5),x=(-170,-120))
nino34 = nino34.weighted(nino34.areacello).mean(("yh","xh"))
nino34 = nino34.load()

In [35]:
# Figure
result = xw.Wavelet(nino34, scaled=True)
fig = result.composite()